In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import pandas as pd
import xgboost
import joblib 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from  sklearn.model_selection import GridSearchCV

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
new_train = pd.read_csv('/content/drive/MyDrive/msbd5001-fall2020/new_train_.csv')
new_test = pd.read_csv('/content/drive/My Drive/msbd5001-fall2020/new_test_.csv')

In [24]:
new_train = new_train.drop(['next_hour'],axis=1)
new_test = new_test.drop(['next_hour'],axis=1)

In [25]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14006 entries, 0 to 14005
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        14006 non-null  int64  
 1   datetime          14006 non-null  int64  
 2   year              14006 non-null  int64  
 3   month             14006 non-null  int64  
 4   day               14006 non-null  int64  
 5   hours             14006 non-null  int64  
 6   if_rush_hours     14006 non-null  int64  
 7   last_hour         14006 non-null  int64  
 8   gap_hour          14006 non-null  int64  
 9   day_of_week       14006 non-null  int64  
 10  last_day_of_week  14006 non-null  int64  
 11  gap_day_of_week   14006 non-null  int64  
 12  if_weekends       14006 non-null  int64  
 13  gap_add           14006 non-null  int64  
 14  speed             14006 non-null  float64
dtypes: float64(1), int64(14)
memory usage: 1.6 MB


In [26]:
dev_train_x, dev_test_x,dev_train_y,dev_test_y\
 = train_test_split(new_train.values[:,2:14].astype(int),new_train.values[:,14:].astype(float), test_size = 0.25,random_state = 1)

In [27]:
train_x = new_train.values[:,2:14].astype(int)
train_y = new_train.values[:,14:].astype(float)
test_x = new_test.values[:,2:].astype(int)

In [28]:
#cv_params={}
other_params={'gamma':0.2,'colsample_bytree':1,'subsample':0.9,'colsample_bylevel':1,'base_score':0.3,
              'learning_rate':0.1,'max_depth':7,'min_child_weight':9,'max_delta_step':0,
              'reg_alpha':0.05,'reg_lambda':1,'n_estimators':300,'objective':'reg:squarederror'}
xgb = xgboost.XGBRegressor(**other_params)
#opt = GridSearchCV(xgb,cv_params,scoring='neg_mean_squared_error',cv=5)
#opt.fit(dev_train_x, dev_train_y)
xgb.fit(dev_train_x, dev_train_y)
#print(opt.best_params_)

XGBRegressor(base_score=0.3, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.2,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=7, min_child_weight=9, missing=None, n_estimators=300,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0.05, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.9, verbosity=1)

In [29]:
#dev_predict_y = opt.predict(dev_test_x)
dev_predict_y = xgb.predict(dev_test_x)
mse = mean_squared_error(dev_test_y,dev_predict_y)
print(mse)

12.270495926080168


In [31]:
xgb1 = xgboost.XGBRegressor(**other_params)
xgb1.fit(train_x, train_y)

XGBRegressor(base_score=0.3, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.2,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=7, min_child_weight=9, missing=None, n_estimators=300,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0.05, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.9, verbosity=1)

In [ ]:
joblib.dump(xgb1, '/content/drive/My Drive/msbd5001-fall2020/xgb_14.pkl')

In [32]:
predict_y = xgb1.predict(test_x)


In [33]:
result = predict_y
Submission = pd.read_csv('/content/drive/My Drive/msbd5001-fall2020/sampleSubmission.csv' )
Submission['speed'] = 0
Submission['speed'] = result
Submission.head(3504)

,id,speed
0,0,47.886978
1,1,48.016453
2,2,40.350746
3,3,29.791910
4,4,39.815365
...,...,...
3499,3499,13.564622
3500,3500,25.086124
3501,3501,48.637436
3502,3502,43.583454


In [ ]:
Submission.to_csv('/content/drive/MyDrive/msbd5001-fall2020/result_xgb.csv',index = False)